In [1]:
import os
%pwd

'd:\\Asad\\krish naik projects\\end-to-end-ML-project\\research'

In [2]:
os.chdir("../")
%pwd

'd:\\Asad\\krish naik projects\\end-to-end-ML-project'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_col: str

In [4]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    def __init__(self,config_path = CONFIG_FILE_PATH,params_path = PARAMS_FILE_PATH,schema_path = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        self.schema = read_yaml(schema_path)

        create_directories([self.config.artifacts_root])

    def get_data_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_col=schema.name
        )

        return model_trainer_config


In [6]:
from sklearn.linear_model import ElasticNet
import joblib
import pandas as pd

[2024-03-31 15:19:47,026:INFO:utils:NumExpr defaulting to 8 threads.]


In [10]:
class ModelTrainer:
    def __init__(self,config:ModelTrainerConfig) :
        self.config = config

    def train_model(self):
        train = pd.read_csv(self.config.train_data_path)
        test = pd.read_csv(self.config.test_data_path)

        train_x = train.drop([self.config.target_col],axis=1)
        test_x = test.drop([self.config.target_col],axis=1)

        train_y = train[[self.config.target_col]]
        test_y = test[[self.config.target_col]]

        lr = ElasticNet(alpha=self.config.alpha,l1_ratio=self.config.l1_ratio,random_state=42)
        lr.fit(train_x,train_y)

        joblib.dump(lr,os.path.join(self.config.root_dir,self.config.model_name))

In [11]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_data_model_trainer_config()
    model_trainer = ModelTrainer(model_trainer_config)
    model_trainer.train_model()

except Exception as e:
    raise e

[2024-03-31 15:22:29,523:INFO:common:yaml file:config\config.yaml loaded successfully]
[2024-03-31 15:22:29,526:INFO:common:yaml file:params.yaml loaded successfully]
[2024-03-31 15:22:29,529:INFO:common:yaml file:schema.yaml loaded successfully]
[2024-03-31 15:22:29,531:INFO:common:created directory at: artifacts]
[2024-03-31 15:22:29,655:INFO:common:created directory at: artifacts/model_trainer]
